In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")

In [ ]:
prompt_val = prompt.invoke({"topic": "dog"})
print(prompt_val)

In [ ]:
print(prompt_val.to_messages())

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")
result = model.invoke(prompt_val)
result

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(result)

### Now let´s do this LCEL

In [26]:

prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

basicchain = model | output_parser

In [27]:
basicchain.invoke("hello!")

'Hello! How can I assist you today?'

In [ ]:
chain = prompt | model | output_parser

chain.invoke({"topic": "dog"})

### Retrieval Augmented Generation with LCEL

In [ ]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

embedding_function = OpenAIEmbeddings()

docs = [
    Document(page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}),
    Document(page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [ ]:
retriever.get_relevant_documents("What does the dog want to eat?")

In [ ]:
retriever.invoke("What does the dog want to eat?")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

In [ ]:
from operator import itemgetter

retrieval_chain = (
    {
        "context": (lambda x: x["question"]) | retriever,
        # "question": lambda x: x["question"],
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
retrieval_chain.invoke({"question": "What does the dog like to eat?"})

In [24]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
retrieval_chain.invoke("What does the dog like to eat?")

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(upper=lambda x: x["input"].upper()),
    modified=lambda x: x["input"] * 3,
)

runnable.invoke({"input": "hello!"})

AttributeError: 'int' object has no attribute 'upper'